In [1]:
!pip install --upgrade pip
!pip install transformers datasets accelerate peft bitsandbytes evaluate safetensors pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 

In [2]:
!pip install --upgrade datasets

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [datasets]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
from datasets import load_dataset

train_ds = load_dataset(
    "ahmed-masry/ChartQA",   # ← HF Hub ID, _not_ a path
    split="train[:10%]",
)
val_ds = load_dataset("ahmed-masry/ChartQA", split="val[:10%]")
test_ds = load_dataset("ahmed-masry/ChartQA", split="test[:10%]")

print(len(train_ds), len(val_ds), len(test_ds))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/219M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/314M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/321M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/68.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28299 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1920 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

2830 192 250


In [4]:
from itertools import groupby
from datasets import Dataset

# 1) Gather Q&A strings per chart
grouped = {}
for ex in train_ds:
    qa = f"{ex['query']} {ex['label'][0]}"
    grouped.setdefault(ex["imgname"], []).append(qa)

# 2) Build a list of dicts with image + combined caption
caption_examples = []
for imgname, qas in grouped.items():
    # find a matching example to get the PIL image
    img = next(e["image"] for e in train_ds if e["imgname"] == imgname)
    caption_examples.append({
        "imgname": imgname,
        "image": img,
        "caption": " ".join(qas)
    })

# 3) Wrap into a new HF Dataset
caption_ds = Dataset.from_list(caption_examples)
print(caption_ds.column_names, len(caption_ds))

['imgname', 'image', 'caption'] 1415


In [5]:
from transformers import BlipProcessor, BlipForConditionalGeneration

In [6]:
MODEL_NAME = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(MODEL_NAME)
model     = BlipForConditionalGeneration.from_pretrained(MODEL_NAME)
model = model.to("cuda")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [9]:
from PIL import Image
import io

def preprocess_fn(ex):
    # Convert image bytes to a PIL Image
    img = Image.open(io.BytesIO(ex["image"]))

    inputs = processor(
        images=img,
        text=ex["caption"],
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    # Trainer expects labels
    inputs["labels"] = inputs.input_ids.clone()
    # squeeze out the batch dim so map works
    return {k: v.squeeze() for k,v in inputs.items()}

# Map over the dataset
train_tok = caption_ds.map(
    preprocess_fn,
    batched=False,
    remove_columns=caption_ds.column_names
)
print(train_tok)

Map:   0%|          | 0/1415 [00:00<?, ? examples/s]

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1415
})


In [19]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./chart_caption_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,

    # old names:
    do_eval=True,
    eval_steps=500,

    logging_steps=50,

    save_steps=500,

    learning_rate=5e-5,
    fp16=True,
)

In [26]:
model.save_pretrained("./chart_caption_model")
processor.save_pretrained("./chart_caption_model")

[]

In [64]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# 1) Load processor & model
processor = BlipProcessor.from_pretrained("./chart_caption_model")
model     = BlipForConditionalGeneration.from_pretrained("./chart_caption_model")
device    = "cuda" if torch.cuda.is_available() else "cpu"
model     = model.to(device)

# 2) Inference helper with a text prompt
def make_caption(img_path, prompt: str = "", max_new_tokens: int = 100) -> str:
    # — vision side —
    img = Image.open(img_path).convert("RGB")
    vision_inputs = processor(images=img, return_tensors="pt").to(device)

    # — text side —
    prompt_tokens = processor.tokenizer(
        prompt,
        add_special_tokens=True,
        return_tensors="pt"
    ).to(device).input_ids

    # generate, but pass prompt tokens under `input_ids` (not decoder_input_ids)
    output_ids = model.generate(
        pixel_values=vision_inputs.pixel_values,
        input_ids=prompt_tokens,        # <— rename decoder_input_ids → input_ids
        max_new_tokens=max_new_tokens,
        num_beams=5,
        early_stopping=True
    )

    return processor.decode(output_ids[0], skip_special_tokens=True)

# 3) Examples of calling it

# a) Zero-prompt (just what it was trained to do):
print(make_caption("/content/charts/line-chart.png"))

a line graph showing the number of wild populations
